# Querying Google Cloud Platform from your computer

## Setup (skip if you have already done this)

These are the steps to set things up to be able to do everything in Cursor or visual studio code etc.


### Install Google Cloud Platform SDK

 1) If you don't have Chocolatey installed, you need to:
 
 - Modify your computer settings to become a temporary admin. **DO NOT PROCEED UNTIL YOU DO THIS!!**
 
 - Open an administrator powershell terminal and do the following:

            ```bash
            Set-ExecutionPolicy Bypass -Scope Process -Force; `
            iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))
            ```

 1) Go to a powershell terminal and type: 
 
 ```bash
 choco install gcloudsdk -y
 ```

 3) Refresh your terminal and then initiate the google cloud platform SDK by typing in the powershell:

 ```bash
 refreshenv
 gcloud init
 ```

 
"gcloud init" is the interactive first-time setup and will have you run through a bunch of steps. Once you do all of those, move on to the next step

### Create or activate your virtual environment

Next, you need to create or activate the virtual environment. Do to that, run the code below:

In [ ]:
import sys
import shutil
from pathlib import Path

# check for existing venv, and use it if it exists. Otherwise, create fresh .venv
venv_path = Path(".venv")
if venv_path.exists():
    print("Using the existing virtual environment...")
  #  shutil.rmtree(venv_path)

else:
    print("Creating fresh virtual environment...")
    !python -m venv .venv --clear

# Install packages using explicit commands
print("\nInstalling packages...")
%pip install --upgrade pip
%pip install requests
%pip install -r requirements.txt
# Show what's installed
print("\nInstalled packages:")
%pip list


### Log in to Google cloud platform

In [ ]:
# If you have installed the Google cloud platform SDK:

!gcloud auth application-default login

# If you do not have your default project set up, set it up here by uncommenting the line below:
#!gcloud config set project aif-usr-p-ent-ai-misc-3444

## Query a table to see what the data looks like and if it's what you want

## First, get all of your tables

In [2]:
#!gcloud auth application-default login
import pandas as pd
import os
from google.cloud import bigquery
from tqdm.notebook import tqdm
import time
import uuid
from datetime import datetime

# Initialize BigQuery client
client = bigquery.Client(project="aif-usr-p-ent-ai-misc-3444")


# Set your target project and dataset
PROJECT_ID = 'aif-usr-p-ent-ai-misc-3444'  # The GCP project ID
DATASET_ID = 'column_analysis'  # The dataset name where results are stored
LOCATION = 'US'  
FINAL_TABLE = 'mayo_database_summary_table'


#

## Query columns to see what their values are with frequency breakdowns

### First, run the code below:

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
import pandas as pd

# Define database aliases and their mappings (reuse from previous code)
DATABASE_ALIASES = {
    "ml-mps-adl-intudp-phi-p-d5cb": "ADL Data Assets",
    "ml-mps-adl-intlhr-phi-p-3b6e": "LPR (FHIR)"
}

# CSS for styling (existing CSS plus new styles for table summary)
display(HTML("""
<style>
    .column-explorer-container {
        background-color: #f8f9fa;
        padding: 20px;
        border-radius: 10px;
        margin: 10px 0;
    }

    .value-table {
        margin-top: 20px;
        border-collapse: collapse;
        width: 100%;
    }

    .value-table th, .value-table td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
    }

    .value-table th {
        background-color: #f0f0f0;
        font-weight: bold;
    }

    .value-table tr:nth-child(even) {
        background-color: #f9f9f9;
    }

    .export-buttons {
        margin-top: 10px;
    }

    .search-info {
        color: #666;
        font-size: 0.9em;
        margin-top: 5px;
    }

    .slider-info {
        color: #666;
        font-size: 0.9em;
        margin-top: 5px;
    }

    .warning-text {
        color: #dc3545;
        font-weight: bold;
        font-size: 0.9em;
        margin-top: 5px;
    }
    
    .tab-content {
        padding: 15px;
        border: 1px solid #ddd;
        border-top: none;
    }
    
    .table-summary-header {
        background-color: #e9ecef;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 15px;
    }
    
    .column-summary-card {
        border: 1px solid #ddd;
        border-radius: 5px;
        margin-bottom: 15px;
        padding: 10px;
    }
    
    .column-summary-header {
        background-color: #f0f0f0;
        padding: 8px;
        border-radius: 5px 5px 0 0;
        font-weight: bold;
    }
</style>
"""))

# Query metadata to get all columns with their tables
print("Loading column information...")
column_metadata_query = f"""
SELECT DISTINCT
    database_name,
    table_schema,
    table_name,
    column_name,
    data_type
FROM `{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE}`
ORDER BY table_name, column_name
"""

column_metadata_df = client.query(column_metadata_query).to_dataframe()

# Create column display names in format {table}.{column}
column_display_names = []
column_info_map = {}

for _, row in column_metadata_df.iterrows():
    display_name = f"{row['table_name']}.{row['column_name']}"
    column_display_names.append(display_name)

    # Store full info for each column
    column_info_map[display_name] = {
        'database_name': row['database_name'],
        'table_schema': row['table_schema'],
        'table_name': row['table_name'],
        'column_name': row['column_name'],
        'data_type': row['data_type']
    }

# NEW: Extract unique tables for table selection dropdown
unique_tables = column_metadata_df[['database_name', 'table_schema', 'table_name']].drop_duplicates()
table_options = []
table_info_map = {}

for _, row in unique_tables.iterrows():
    table_display_name = row['table_name']
    table_options.append(table_display_name)
    
    # Store full info for each table
    table_info_map[table_display_name] = {
        'database_name': row['database_name'],
        'table_schema': row['table_schema'],
        'table_name': row['table_name']
    }

# Create tabs for single column vs table modes
mode_tabs = widgets.Tab()
single_column_tab = widgets.VBox()
table_summary_tab = widgets.VBox()
mode_tabs.children = [single_column_tab, table_summary_tab]
mode_tabs.set_title(0, 'Single Column Explorer')
mode_tabs.set_title(1, 'Table Summary Explorer')

# SINGLE COLUMN EXPLORER WIDGETS (existing)
column_search_widget = widgets.Text(
    placeholder='Search for column name...',
    description='Column Search:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

column_dropdown = widgets.Dropdown(
    options=column_display_names,
    value=column_display_names[0] if column_display_names else '',
    description='Select Column:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

result_limit_slider = widgets.IntSlider(
    value=100,
    min=20,
    max=10000,
    step=10,
    description='Download Limit:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

no_limit_checkbox = widgets.Checkbox(
    value=False,
    description='No limit (download all values)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

slider_value_text = widgets.HTML(
    value=f'<div class="slider-info">Will download top {result_limit_slider.value} values (display shows top 20)</div>'
)

warning_text = widgets.HTML(value='')

get_values_button = widgets.Button(
    description="Get Column Values",
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

# Output areas for single column mode
search_info = widgets.HTML(value='')
output_area = widgets.Output()
export_area = widgets.Output()

# TABLE SUMMARY EXPLORER WIDGETS (new)
table_search_widget = widgets.Text(
    placeholder='Search for table name...',
    description='Table Search:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

table_dropdown = widgets.Dropdown(
    options=table_options,
    value=table_options[0] if table_options else '',
    description='Select Table:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

table_search_info = widgets.HTML(value='')

# Number of top values to show for each column in table mode
table_top_values_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description='Top Values Per Column:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

get_table_summary_button = widgets.Button(
    description="Get Table Summary",
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

# Output area for table summary mode
table_summary_output = widgets.Output()
table_export_area = widgets.Output()

# Update slider value text when slider changes or checkbox changes (existing function)
def update_ui_text(change=None):
    if no_limit_checkbox.value:
        slider_value_text.value = '<div class="slider-info">Will download ALL values (display shows top 20)</div>'
        warning_text.value = '<div class="warning-text">⚠️ Warning: Downloading all values may take a long time and produce large files for columns with many unique values!</div>'
        result_limit_slider.disabled = True
    else:
        slider_value_text.value = f'<div class="slider-info">Will download top {result_limit_slider.value} values (display shows top 20)</div>'
        warning_text.value = ''
        result_limit_slider.disabled = False

result_limit_slider.observe(update_ui_text, names='value')
no_limit_checkbox.observe(update_ui_text, names='value')

# Function to filter columns based on search (existing)
def filter_columns(change):
    """Filter columns based on search text"""
    search_text = change['new'].lower()

    if search_text:
        # Filter columns that contain the search text
        filtered_columns = [
            col for col in column_display_names
            if search_text in col.lower()
        ]

        # Update dropdown options
        column_dropdown.options = filtered_columns

        # Update search info
        search_info.value = f'<div class="search-info">Found {len(filtered_columns)} columns matching "{search_text}"</div>'

        # Set first value if available
        if filtered_columns:
            column_dropdown.value = filtered_columns[0]
    else:
        # Reset to all columns
        column_dropdown.options = column_display_names
        search_info.value = ''

# NEW: Function to filter tables based on search
def filter_tables(change):
    """Filter tables based on search text"""
    search_text = change['new'].lower()

    if search_text:
        # Filter tables that contain the search text
        filtered_tables = [
            table for table in table_options
            if search_text in table.lower()
        ]

        # Update dropdown options
        table_dropdown.options = filtered_tables

        # Update search info
        table_search_info.value = f'<div class="search-info">Found {len(filtered_tables)} tables matching "{search_text}"</div>'

        # Set first value if available
        if filtered_tables:
            table_dropdown.value = filtered_tables[0]
    else:
        # Reset to all tables
        table_dropdown.options = table_options
        table_search_info.value = ''

# Store download data globally for access by download functions
download_data_df = None
table_summary_data = None

# Existing function to get column values with counts
def get_column_values(button):
    global download_data_df

    with output_area:
        clear_output()

        selected_column = column_dropdown.value
        if not selected_column:
            print("Please select a column")
            return

        # Get column info
        info = column_info_map[selected_column]

        # Build full table name
        full_table_name = f"{info['database_name']}.{info['table_schema']}.{info['table_name']}"

        # Query to get value counts for display (always top 20)
        display_query = f"""
        SELECT
            `{info['column_name']}` as value,
            COUNT(*) as count
        FROM `{full_table_name}`
        GROUP BY `{info['column_name']}`
        ORDER BY count DESC
        LIMIT 20
        """

        # Query to get value counts for download (with or without limit)
        if no_limit_checkbox.value:
            download_query = f"""
            SELECT
                `{info['column_name']}` as value,
                COUNT(*) as count
            FROM `{full_table_name}`
            GROUP BY `{info['column_name']}`
            ORDER BY count DESC
            """
            download_limit_text = "ALL"
        else:
            download_query = f"""
            SELECT
                `{info['column_name']}` as value,
                COUNT(*) as count
            FROM `{full_table_name}`
            GROUP BY `{info['column_name']}`
            ORDER BY count DESC
            LIMIT {result_limit_slider.value}
            """
            download_limit_text = f"top {result_limit_slider.value}"

        try:
            # Get database display name
            db_display = DATABASE_ALIASES.get(info['database_name'], info['database_name'])

            print(f"Fetching values for: {selected_column}")
            print(f"Database: {db_display}")
            print(f"Table: {info['table_name']}")
            print(f"Column: {info['column_name']}")
            print(f"Data Type: {info['data_type']}")
            print("-" * 50)

            # Execute both queries
            display_df = client.query(display_query).to_dataframe()

            print(f"Downloading {download_limit_text} values...")
            download_data_df = client.query(download_query).to_dataframe()

            # Print dataframe dimensions
            print(f"Downloaded dataframe shape: {download_data_df.shape[0]} rows × {download_data_df.shape[1]} columns")

            print(f"Displaying top 20 values (will download {download_limit_text} values)")

            # Create styled table for display
            table_html = f"""
            <h3>Top 20 Values in {selected_column}</h3>
            <p style='color: #666; margin-bottom: 10px;'>
                Total unique values found: <strong>{download_data_df.shape[0]}</strong>
                (downloading {download_limit_text} values)
            </p>
            <table class='value-table'>
                <thead>
                    <tr>
                        <th>Value</th>
                        <th>Count</th>
                    </tr>
                </thead>
                <tbody>
            """

            for _, row in display_df.iterrows():
                value = str(row['value']) if row['value'] is not None else '<NULL>'
                count = row['count']
                table_html += f"""
                    <tr>
                        <td>{value}</td>
                        <td>{count:,}</td>
                    </tr>
                """

            table_html += """
                </tbody>
            </table>
            """

            display(HTML(table_html))

            # Export functionality
            with export_area:
                clear_output()

                export_buttons = widgets.HBox([
                    widgets.Button(description="Download CSV", button_style='success'),
                    widgets.Button(description="Download Excel", button_style='success')
                ])

                # Status message widget
                status_message = widgets.HTML(value="")
                download_output = widgets.Output()

                def download_csv(b):
                    with download_output:
                        clear_output()
                        try:
                            # Create a filename based on the selected column
                            limit_suffix = "all" if no_limit_checkbox.value else f"{result_limit_slider.value}"
                            filename = f"{selected_column.replace('.', '_')}_values_{limit_suffix}.csv"

                            # Update status message
                            status_message.value = '<div style="color: #666;">Saving CSV file...</div>'

                            # Save to current working directory (using download_data_df)
                            download_data_df.to_csv(filename, index=False)

                            # Create download link using FileLink
                            display(FileLink(filename))

                            # Update status message with success
                            status_message.value = f'<div style="color: #28a745; font-weight: bold;">✅ CSV file saved: {filename} ({len(download_data_df)} rows)</div>'

                        except Exception as e:
                            # Update status message with error
                            status_message.value = f'<div style="color: #dc3545; font-weight: bold;">❌ Error saving CSV: {e}</div>'

                def download_excel(b):
                    with download_output:
                        clear_output()
                        try:
                            # Create a filename based on the selected column
                            limit_suffix = "all" if no_limit_checkbox.value else f"{result_limit_slider.value}"
                            filename = f"{selected_column.replace('.', '_')}_values_{limit_suffix}.xlsx"

                            # Update status message
                            status_message.value = '<div style="color: #666;">Saving Excel file...</div>'

                            # Save to current working directory using ExcelWriter (using download_data_df)
                            with pd.ExcelWriter(filename, engine="openpyxl") as writer:
                                download_data_df.to_excel(writer, index=False, sheet_name="Column Values")

                            # Create download link using FileLink
                            display(FileLink(filename))

                            # Update status message with success
                            status_message.value = f'<div style="color: #28a745; font-weight: bold;">✅ Excel file saved: {filename} ({len(download_data_df)} rows)</div>'

                        except Exception as e:
                            # Update status message with error
                            status_message.value = f'<div style="color: #dc3545; font-weight: bold;">❌ Error saving Excel: {e}</div>'

                export_buttons.children[0].on_click(download_csv)
                export_buttons.children[1].on_click(download_excel)

                display(HTML('<div class="export-buttons"><h4>Export Options</h4></div>'))
                display(export_buttons)
                display(status_message)
                display(download_output)

        except Exception as e:
            print(f"Error: {e}")

# NEW: Function to get summaries for all columns in a table
def get_table_summary(button):
    global table_summary_data
    
    with table_summary_output:
        clear_output()
        
        selected_table = table_dropdown.value
        if not selected_table:
            print("Please select a table")
            return
        
        # Get table info
        info = table_info_map[selected_table]
        
        # Build full table name
        full_table_name = f"{info['database_name']}.{info['table_schema']}.{info['table_name']}"
        
        # Get all columns for this table
        table_columns = [
            col_info for col_name, col_info in column_info_map.items()
            if col_info['table_name'] == selected_table
        ]
        
        # Get database display name
        db_display = DATABASE_ALIASES.get(info['database_name'], info['database_name'])
        
        # Display table info header
        print(f"Generating summary for table: {selected_table}")
        print(f"Database: {db_display}")
        print(f"Schema: {info['table_schema']}")
        print(f"Number of columns: {len(table_columns)}")
        print("-" * 50)
        
        # Create a dictionary to store all column summaries
        all_column_summaries = {}
        
        # Get top values limit from slider
        top_values_limit = table_top_values_slider.value
        
        # Process each column
        for i, column_info in enumerate(table_columns):
            column_name = column_info['column_name']
            
            print(f"Processing column {i+1}/{len(table_columns)}: {column_name}")
            
            # Query to get value counts for this column
            column_query = f"""
            SELECT
                `{column_name}` as value,
                COUNT(*) as count
            FROM `{full_table_name}`
            GROUP BY `{column_name}`
            ORDER BY count DESC
            LIMIT {top_values_limit}
            """
            
            try:
                # Execute query
                column_df = client.query(column_query).to_dataframe()
                
                # Store in our summaries dictionary
                all_column_summaries[column_name] = {
                    'data_type': column_info['data_type'],
                    'unique_values': len(column_df),
                    'values': column_df.to_dict('records')
                }
                
            except Exception as e:
                print(f"Error processing column {column_name}: {e}")
                all_column_summaries[column_name] = {
                    'data_type': column_info['data_type'],
                    'unique_values': 0,
                    'error': str(e),
                    'values': []
                }
        
        # Store the summary data for potential export
        table_summary_data = all_column_summaries
        
        # Generate HTML to display all column summaries
        summary_html = f"""
        <div class="table-summary-header">
            <h2>Summary for Table: {selected_table}</h2>
            <p>Database: {db_display} | Schema: {info['table_schema']} | Total Columns: {len(table_columns)}</p>
        </div>
        """
        
        # Add each column's summary
        for column_name, summary in all_column_summaries.items():
            summary_html += f"""
            <div class="column-summary-card">
                <div class="column-summary-header">
                    {column_name} ({summary['data_type']})
                </div>
            """
            
            if 'error' in summary:
                summary_html += f"""
                <div style="color: #dc3545; padding: 8px;">
                    Error: {summary['error']}
                </div>
                """
            else:
                summary_html += f"""
                <div style="padding: 8px;">
                    <p>Top {len(summary['values'])} values (out of {summary['unique_values']} unique values):</p>
                    <table class="value-table">
                        <thead>
                            <tr>
                                <th>Value</th>
                                <th>Count</th>
                            </tr>
                        </thead>
                        <tbody>
                """
                
                for value_data in summary['values']:
                    value = str(value_data['value']) if value_data['value'] is not None else '<NULL>'
                    count = value_data['count']
                    summary_html += f"""
                            <tr>
                                <td>{value}</td>
                                <td>{count:,}</td>
                            </tr>
                    """
                
                summary_html += """
                        </tbody>
                    </table>
                </div>
                """
            
            summary_html += """
            </div>
            """
        
        display(HTML(summary_html))
        
        # Add export functionality for table summary
        with table_export_area:
            clear_output()
            
            table_export_buttons = widgets.HBox([
                widgets.Button(description="Download Summary Excel", button_style='success')
            ])
            
            # Status message widget
            table_status_message = widgets.HTML(value="")
            table_download_output = widgets.Output()
            
            def download_table_summary_excel(b):
                with table_download_output:
                    clear_output()
                    try:
                        # Create a filename
                        filename = f"{selected_table}_column_summary.xlsx"
                        
                        # Update status message
                        table_status_message.value = '<div style="color: #666;">Creating Excel summary file...</div>'
                        
                        # Create Excel writer
                        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
                            # Add a sheet with table overview
                            overview_df = pd.DataFrame({
                                'Column Name': [col_name for col_name in all_column_summaries.keys()],
                                'Data Type': [summary['data_type'] for summary in all_column_summaries.values()],
                                'Unique Values': [summary.get('unique_values', 0) for summary in all_column_summaries.values()]
                            })
                            overview_df.to_excel(writer, sheet_name="Overview", index=False)
                            
                            # Add a sheet for each column
                            for column_name, summary in all_column_summaries.items():
                                if 'error' not in summary and summary['values']:
                                    # Create dataframe from values
                                    col_df = pd.DataFrame(summary['values'])
                                    
                                    # Safe sheet name (Excel has a 31 character limit)
                                    sheet_name = column_name[:30]
                                    
                                    # Write to Excel
                                    col_df.to_excel(writer, sheet_name=sheet_name, index=False)
                        
                        # Create download link
                        display(FileLink(filename))
                        
                        # Update status message
                        table_status_message.value = f'<div style="color: #28a745; font-weight: bold;">✅ Excel summary saved: {filename}</div>'
                        
                    except Exception as e:
                        # Update status message with error
                        table_status_message.value = f'<div style="color: #dc3545; font-weight: bold;">❌ Error saving Excel: {e}</div>'
            
            table_export_buttons.children[0].on_click(download_table_summary_excel)
            
            display(HTML('<div class="export-buttons"><h4>Export Table Summary</h4></div>'))
            display(table_export_buttons)
            display(table_status_message)
            display(table_download_output)

# Attach event handlers
column_search_widget.observe(filter_columns, names='value')
table_search_widget.observe(filter_tables, names='value')
get_values_button.on_click(get_column_values)
get_table_summary_button.on_click(get_table_summary)

# Create the limit controls for single column tab
limit_controls = widgets.HBox([
    result_limit_slider,
    no_limit_checkbox
], layout=widgets.Layout(margin='10px 0'))

# Assemble Single Column Tab
single_column_tab.children = [
    widgets.HTML("""
    <div class="column-explorer-container">
        <h2>📊 Single Column Value Explorer</h2>
        <p>Search for a specific column and view its value distribution</p>
    </div>
    """),
    widgets.VBox([
        column_search_widget,
        search_info,
        column_dropdown,
        limit_controls,
        slider_value_text,
        warning_text,
        get_values_button
    ], layout=widgets.Layout(margin='10px')),
    output_area,
    export_area
]

# Assemble Table Summary Tab
table_summary_tab.children = [
    widgets.HTML("""
    <div class="column-explorer-container">
        <h2>📈 Table Column Summary Explorer</h2>
        <p>Select a table to view summaries of all its columns at once</p>
    </div>
    """),
    widgets.VBox([
        table_search_widget,
        table_search_info,
        table_dropdown,
        table_top_values_slider,
        get_table_summary_button
    ], layout=widgets.Layout(margin='10px')),
    table_summary_output,
    table_export_area
]

# Display the complete interface with tabs
display(mode_tabs)